# Amazon Aurora CloudWatch Integration Tutorial

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#Setting-up-the-environment-and-required-libraries">Setting up the environment and required libraries</a></li>
<li><a href="#1-Setting-up-Custom-Metrics">1. Setting up Custom Metrics</a></li>
<li><a href="#2-Creating-CloudWatch-Dashboard">2. Creating CloudWatch Dashboard</a></li>
<li><a href="#3-Configuring-Log-Streams">3. Configuring Log Streams</a></li>
<li><a href="#4-Enhanced-Monitoring-Setup">4. Enhanced Monitoring Setup</a></li>
<li><a href="#5-Get-Enhanced-Monitoring-Metrics">5. Get Enhanced Monitoring Metrics</a></li>
<li><a href="#Usage-Example">Usage Example</a></li>
</ul>
</div>

## Prerequisites

Before starting this workshop, ensure you have:

- ✅ **Jupyter Notebook**: You can launch a [free tier Amazon SageMaker Jupyter Notebook](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)
- ✅ **Aurora PostgreSQL Cluster**: An active Aurora PostgreSQL cluster running in your AWS account
  - If you don't have one, follow the setup guide: [Your First Database on AWS](../../2_Your_First_Database_on_AWS/README.MD)
- ✅ **AWS CLI configured** with appropriate permissions for RDS and CloudWatch
- ✅ **IAM permissions** for:
  - CloudWatch metrics and dashboards
  - Aurora monitoring and modification
  - Enhanced Monitoring role creation

> 💡 **Note**: This workshop uses Aurora PostgreSQL examples, but the monitoring concepts apply to Aurora MySQL as well.

## Setting up the environment and required libraries

Initialize required AWS clients for CloudWatch and Aurora operations like AWS SDK functionality, pandas for data manipulation, and datetime for time-based operations. The AWS clients will be initialized for CloudWatch metrics and Aurora PostgreSQL management.

In [ ]:
# Replace with your Aurora instance identifier
CLUSTER_IDENTIFIER='<cluster-identifier>' #use the cluster name created on 2.1.2 section
MONITORING_ROLE_ARN='<role-identifier>' #use the role created on 2.1.2 section

## 1. Setting up Custom Metrics

Create custom CloudWatch metrics for Aurora monitoring to track application-specific metrics alongside standard AWS metrics. The function publishes a custom connection count metric with proper dimensions for database instance identification and returns the metric data structure with publication response.

In [ ]:
%%bash
export CLUSTER_IDENTIFIER='<cluster-identifier>'

aws cloudwatch put-metric-data \
    --namespace "Custom/Aurora" \
    --metric-data \
    '[{
        "MetricName": "CustomConnectionCount",
        "Value": 1,
        "Unit": "Count",
        "Dimensions": [{
            "Name": "DBInstanceIdentifier",
            "Value": "'${CLUSTER_IDENTIFIER}'"
        }]
    }]'

## 2. Creating CloudWatch Dashboard

Create a comprehensive CloudWatch dashboard for Aurora instance monitoring that displays key metrics like CPU utilization and memory usage in visual format. The dashboard configuration includes metric specifications, time periods, and regional settings for optimal monitoring with access through the CloudWatch console.

In [ ]:
%%bash
export CLUSTER_IDENTIFIER='<cluster-identifier>'

aws cloudwatch put-dashboard \
    --dashboard-name "Aurora-Dashboard-$CLUSTER_IDENTIFIER" \
    --dashboard-body "{
        \"widgets\": [
            {
                \"type\": \"metric\",
                \"properties\": {
                    \"metrics\": [
                        [\"AWS/RDS\", \"CPUUtilization\", \"DBInstanceIdentifier\", \"${CLUSTER_IDENTIFIER}\"],
                        [\"AWS/RDS\", \"FreeableMemory\", \"DBInstanceIdentifier\", \"${CLUSTER_IDENTIFIER}\"]
                    ],
                    \"period\": 300,
                    \"stat\": \"Average\",
                    \"region\": \"us-east-1\",
                    \"title\": \"Aurora Instance Metrics\"
                }
            }
        ]
    }"

## 3. Configuring Log Streams

Enable CloudWatch log exports for Aurora to stream database logs including error, general, slow query, and audit logs to CloudWatch for centralized monitoring and analysis. The configuration modifies the DB instance to export specified log types automatically with confirmation response. That example is focused on Aurora PostgreSQL cluster, for another engine types look: https://docs.aws.amazon.com/cli/latest/reference/rds/modify-db-cluster.html

In [ ]:
%%bash
echo "Configuring Log Export..."
export CLUSTER_IDENTIFIER='<cluster-identifier>'
export MONITORING_ROLE_ARN='<monitoring-role>'

aws rds modify-db-cluster \
    --db-cluster-identifier $CLUSTER_IDENTIFIER \
    --cloudwatch-logs-export-configuration '{"EnableLogTypes":["instance","postgresql"]}'

## 4. Enhanced Monitoring Setup

Enable Enhanced Monitoring for Aurora instances to provide detailed operating system metrics at 60-second intervals through a dedicated monitoring role. Enhanced Monitoring offers deeper insights into database performance including process lists, memory usage, and I/O statistics with configuration response showing interval and role ARN.

In [ ]:
%%bash
echo "Configuring Enhaced Monitoring settings..."
export CLUSTER_IDENTIFIER='<cluster-identifier>'
export MONITORING_ROLE_ARN='<monitoring-role>'

aws rds modify-db-cluster \
    --db-cluster-identifier ${CLUSTER_IDENTIFIER} \
    --monitoring-interval 60 \
    --monitoring-role-arn ${MONITORING_ROLE_ARN}

## 5. Get Enhanced Monitoring Metrics

Retrieve Enhanced Monitoring metrics from CloudWatch focusing on OS-level process information for detailed performance analysis. The query fetches metrics over the past hour with 60-second granularity to provide comprehensive system insights and returns metric statistics with process list data.

In [ ]:
%%bash
echo "Getting Enhaced Monitoring metrics..."

export CLUSTER_IDENTIFIER='<cluster-identifier>'

# Calculate time stamps (1 hour ago to now)
END_TIME=$(date -u +"%Y-%m-%dT%H:%M:%SZ")
START_TIME=$(date -u -v-1H +"%Y-%m-%dT%H:%M:%SZ")  # For macOS
# For Linux use: START_TIME=$(date -u -d '1 hour ago' +"%Y-%m-%dT%H:%M:%SZ")

aws cloudwatch get-metric-statistics \
    --namespace "AWS/RDS" \
    --metric-name "OSMemoryTotal" \
    --dimensions Name=DBClusterIdentifier,Value=${CLUSTER_IDENTIFIER} \
    --start-time ${START_TIME} \
    --end-time ${END_TIME} \
    --period 60 \
    --statistics Average

## Next Steps 🚀

Now that you've set up comprehensive monitoring for your Aurora cluster, continue building operational excellence with these next steps:

1. **Set Up Alerting and Notifications**
   - Configure CloudWatch alarms for critical metrics in [4.2 Alerting Setup](../4.2_Alerting_Setup/alert_setup.ipynb)
   - Learn how to create proactive monitoring and incident response

2. **Implement Automated Backup Solutions**
   - Set up automated backup strategies in [4.3 Automated Backup Solutions](../4.3_Automated_Backup_Solutions/automate_backup.ipynb)
   - Configure point-in-time recovery and cross-region backups

3. **Optimize Database Performance**
   - Use performance optimization tools in [4.4 Performance Optimization Tools](../4.4_Performance_Optimization_Tools/performance_optimization.ipynb)
   - Learn query optimization and resource tuning techniques

4. **Set Up Development and Testing Environments**
   - Implement Aurora cloning for testing in [4.5 Aurora Cloning for Testing and Development](../4.5_Aurora_Cloning_for_Testing_and_Development/aurora_cloning_testing_devlop.ipynb)
   - Create isolated environments for safe testing and development

Continue to [4.2 Set Up Alerting and Notifications](../4.2_Alerting_Setup/alert_setup.ipynb).


## Additional Resources 📚

### CloudWatch Integration
- [CloudWatch Metrics for Aurora](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/MonitoringAurora.html)
- [Enhanced Monitoring](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_Monitoring.OS.html)
- [CloudWatch Dashboards](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/CloudWatch_Dashboards.html)

### Logging & Analysis
- [Aurora Log Types](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_LogAccess.html)
- [CloudWatch Logs](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/WhatIsCloudWatchLogs.html)
- [Log Analysis Best Practices](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/AnalyzingLogData.html)

### Performance Monitoring
- [Performance Insights](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_PerfInsights.html)
- [Aurora Monitoring Best Practices](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.BestPractices.html)
- [Custom Metrics Guide](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/publishingMetrics.html)